# Objective of this notebook:
1. Explain Notebook
2. Serve as stepping stone to allow function to be reproduced across TS Pred Domains

### Competition Explanation
1. Leaderboard score: 6476
2. Notebook Score: 9766, which served as baseline and building block to notebook of score: 11,480
3. Notebook Credit: https://www.kaggle.com/code/tarlannazarov/own-jane-street-with-keras-nn/notebook

#### From : 1_MLP Model_Part 3 - Global Function

In [1]:
# Print Tensorflow, Keras, Numpy version to make version clear to prevent dependecy issues
import tensorflow
import keras
import numpy 
print(tensorflow.__version__)
print(keras.__version__)
print(numpy.__version__)

2.7.0-dev20210806
2.6.0
1.19.5


In [37]:
from tensorflow.keras.layers import Input, Dense, BatchNormalization, Dropout, Concatenate, Lambda, GaussianNoise, Activation
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers.experimental.preprocessing import Normalization
import tensorflow as tf
import numpy as np
import pandas as pd
from tqdm import tqdm
from random import choices

class Kaggle_custom():
    def __init__(self, n_jobs=-1, verbose=0):
        self.n_jobs = n_jobs # -1: all CPUs are used
        self.verbose = verbose

    def JaneStreet_MLP_model_1(self, df, date, target_list, features_list,
                            
                            train_test_split_size,
                            fillna_type,
                            
                            batch_size,
                            hidden_units,
                            dropout_rates,
                            label_smoothing,
                            learning_rate,
                            epochs,                            
                            ):
        
        '''
        format:
        :param Parameter: [type]: {Example or Explanation}
        
        :param df: [pandas DataFrame]: {DataFrame}
        :param date: Not Required, for future input reference
        :param target_list: [List] : {target(s) column list}
        :param features_list: [List] : {features column list}

        :param batch_size: [int] : {5000}
        :param hidden_units: [List] : {[150, 150, 150]}
        :param dropout_rates: [List] : {[0.2, 0.2, 0.2, 0.2]}
        :param label_smoothing: [exponential notation] : {1e-2}
        :param learning_rate: [exponential notation] : {1e-3}
        :param epochs: [int] : {2000}   
        
        '''
        
        print('[Warning] This is a MULTI VARIATE Time Series Prediction')

        from sklearn.model_selection import train_test_split
        #Param PreProcessing
        train, test = train_test_split(df, test_size=train_test_split_size, shuffle=False)
        features = features_list
        
        if fillna_type == 'mean':
            train.fillna(train.mean(),inplace=True)
        else:
            #input custom lambda function or fill_value function
            fill_value = train.fillna(fill_value, inplace=True)

        train['action'] = ((train['resp'].values) > 0).astype(int)

#         f_mean = np.mean(train[features[1:]].values,axis=0) #Original
        f_mean = np.mean(train[features].values,axis=0) # preferred

        X_train = train.loc[:, train.columns.str.contains('feature')]
        y_train = np.stack([(train[c] > 0).astype('int') for c in target_list]).T

        def create_mlp(
            num_columns, num_labels, hidden_units, dropout_rates, label_smoothing, learning_rate
        ):

            inp = tf.keras.layers.Input(shape=(num_columns,))
            x = tf.keras.layers.BatchNormalization()(inp)
            x = tf.keras.layers.Dropout(dropout_rates[0])(x)
            for i in range(len(hidden_units)):
                x = tf.keras.layers.Dense(hidden_units[i])(x)
                x = tf.keras.layers.BatchNormalization()(x)
                x = tf.keras.layers.Activation(tf.keras.activations.swish)(x)
                x = tf.keras.layers.Dropout(dropout_rates[i + 1])(x)

            x = tf.keras.layers.Dense(num_labels)(x)
            out = tf.keras.layers.Activation("sigmoid")(x)

            model = tf.keras.models.Model(inputs=inp, outputs=out)
            model.compile(
                optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                loss=tf.keras.losses.BinaryCrossentropy(label_smoothing=label_smoothing),
                metrics=tf.keras.metrics.AUC(name="AUC"),
            )

            return model

        #Create Model
        clf = create_mlp(len(features), 5, hidden_units, dropout_rates, label_smoothing, learning_rate)
        clf.fit(X_train, y_train, epochs=epochs, batch_size=batch_size)#epochs=200

        models = []
        models.append(clf)

#### Sample Input

In [38]:
# df = pd.read_csv('./Jane Street Data/Splitted_Data/JaneStreet_Part0.csv',index_col = 0)
# features_list = [c for c in df.columns if "feature" in c]
# date = 'date' 
# target_list = 'resp'
# feature = ['feature1','feature2'] 
# target_list = ['resp_1', 'resp_2', 'resp_3', 'resp', 'resp_4']
# train_test_split_size = 0.2
# fillna_type = 'mean'      #Choose fill Type 1 
# # fill_value = train.mean() #Choose fill Type 2

# Kaggle_custom().JaneStreet_MLP_model_1(df, date, target_list, features_list,
                                       
#                                        train_test_split_size=0.2,
#                                        fillna_type=fillna_type,
#                                         batch_size = 5000,
#                                         hidden_units = [150, 150, 150],
#                                         dropout_rates = [0.2, 0.2, 0.2, 0.2],
#                                         label_smoothing = 1e-2,
#                                         learning_rate = 1e-3,
#                                         epochs=2#2000
#                                     )        

[Warning] This is a MULTI VARIATE Time Series Prediction


c:\users\malco\appdata\local\programs\python\python36\lib\site-packages\pandas\core\series.py:4536: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
c:\users\malco\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Epoch 1/2
32/32 [==============================] - 3s 79ms/step - loss: 0.7341 - AUC: 0.5000
Epoch 2/2
32/32 [==============================] - 3s 79ms/step - loss: 0.7107 - AUC: 0.5005
